## 재무제표 데이터를 지표화

In [1]:
import requests
from bs4 import BeautifulSoup
import io
import zipfile
import xmltodict
from io import BytesIO
from zipfile import ZipFile
from xml.etree.ElementTree import parse
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import numpy as np

## 재무상태표
1. 자본총계: 자산총계 - 부채총계

## 손익계산서
1. 매출액: 매출총이익 + 매출원가
2. PSR: 매출액 / 주가
3. GP/A: 매출총이익 / 자산총계


In [2]:
#2. PSR = 당기 공시기점의 종가 / (당기매출액 / 발행주식수) 

# corp_list랑 corp_code_list는 api_final_result 파일에서 가져오면 됨

api_final_result = pd.read_csv('./api_final_result.csv', dtype={'종목코드':str, '주가코드': str})

# 법인등록번호 가져오기
corp_list = ['삼성전자', '삼성생명', '현대자동차', '셀트리온', 'POSCO홀딩스']
corp_code_list = ['00126380', '00126256', '00164742', '00413046', '00155319']
crnos = []
crnos_dict = {}
for corp_code in corp_code_list:
    params_1 = {
        'crtfc_key' : '5b8f0aa6967605a94ba6e9a39b461c2e86cf0d85',
        'corp_code' : corp_code
    }
    r_temp = requests.get('https://opendart.fss.or.kr/api/company.json', params = params_1)
    import json
    crnos.append(json.loads(r_temp.text)['jurir_no'])
    
    crnos_dict[corp_code] = json.loads(r_temp.text)['jurir_no']


#가져온 법인등록번호로 발행주식수 가져오기

URL = "http://apis.data.go.kr/1160100/service/GetStocIssuInfoService/getItemBasiInfo"
psr_dict = {}

for crno in crnos:
    params = {
        'pageNo' : 1,
        'numOfRows' : 1,
        'resultType' : 'json',
        'serviceKey' : '+21XqtAVW3QkzmAlufEeZE+QJaIxraq0IUyEwMLo0KbKkVfrEecTdm2PLj3j4iNwjSWxzydNtkaFlYQluK68jA==',
        'crno' : crno
    }
    r = requests.get(URL, params = params)
    import json
    try: 
        json_obj = json.loads(r.text)['response']['body']['items']['item'][0]
        
        # 키 : 주식발행회사명, 값 : 발행주식수 -> psr_dict에 저장
        psr_dict[json_obj["stckIssuCmpyNm"]] = json_obj['issuStckCnt'] 
    except: 
        print(json_obj["stckIssuCmpyNm"])
        continue


# stock_data에서 날짜에 맞는 종가 가져오기
import datetime
stock_data = pd.read_csv('./stock_data(5y).csv', dtype={'Code':str})
stock_data = stock_data[['Date', 'Code', 'Close']]
stock_data['Date'] = pd.to_datetime(stock_data['Date'])
stock_data['Date'] = stock_data['Date'].dt.strftime('%Y%m%d')


# api_final_result에 stock_data의 기준일자에 맞는 종가를 붙이기
for i in range(len(api_final_result['기준일자'])):
    api_final_result.loc[i, '기준일자'] = api_final_result['기준일자'][i]//1000000


# 분기별 종가를 새 컬럼으로 붙임. 이름은 '분기종가'
api_final_result['분기종가'] = None

for i in range(len(api_final_result)):
    for j in range(len(stock_data)):
        if (str(api_final_result.loc[i, '기준일자']) == stock_data.loc[j, 'Date']) and (str(api_final_result.loc[i, '주가코드']) == stock_data.loc[j, 'Code']):
            api_final_result.loc[i, '분기종가'] = stock_data.loc[j, 'Close']
        else:
            continue


api_final_result #주가코드, 분기별 종가 추가 완료

#발행주식수 추가하자
crnos_list = list(crnos_dict.keys())

api_final_result['발행주식수'] = None

for i in range(len(api_final_result)):
    for j in range(len(crnos_dict.keys())):
        if (api_final_result.loc[i, '종목코드'] == crnos_list[j]):
            api_final_result.loc[i, '발행주식수'] = crnos_dict[crnos_list[j]]


#발행주식수 컬럼 추가 완료.

api_final_result

,기준일자,기업명,종목코드,유형자산의 처분,분기코드,이자의 수취,비유동자산,재고자산,매출총이익,비지배지분,...,유형자산의 취득,무형자산,자산총계,유동자산,무형자산의 취득,사업년도,분기명,주가코드,분기종가,발행주식수
0,20180515,삼성전자,00126380,1.415690e+11,11013,94118000000,132836079000000,8.915860e+12,17410718000000,94546000000,...,7870910000000,2823356000000,205206164000000,72370085000000,218113000000,2018,1분기,005930,49200,1301110006246
1,20180814,삼성전자,00126380,1.982810e+11,11012,222063000000,135899888000000,9.935909e+12,16708550000000,81090000000,...,13744369000000,2857695000000,203970112000000,68070224000000,429125000000,2018,2분기,005930,45150,1301110006246
2,20181114,삼성전자,00126380,2.829450e+11,11014,360840000000,135895097000000,1.035734e+13,20067142000000,178867000000,...,17692885000000,2814997000000,217799444000000,81904347000000,604961000000,2018,3분기,005930,44100,1301110006246
3,20190401,삼성전자,00126380,3.405580e+11,11011,459074000000,138981902000000,1.244095e+13,68715364000000,450142000000,...,21387378000000,2901476000000,219021357000000,80039455000000,880032000000,2018,4분기,005930,45050,1301110006246
4,20190515,삼성전자,00126380,1.054600e+11,11013,98290000000,138335268000000,1.433641e+13,10586172000000,31372000000,...,2821582000000,2876176000000,220646957000000,82311689000000,250866000000,2019,1분기,005930,42550,1301110006246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,20220516,POSCO홀딩스,00155319,-1.448500e+10,11013,38674284562,47458142280986,NaN,2928765425577,214947479400,...,355821595282,13676722585,53333865909937,5875723628951,8668278674,2022,1분기,005490,281000,1746110000741
80,20220816,POSCO홀딩스,00155319,-1.448500e+10,11012,62434274729,47422658774188,NaN,2733139872051,254454407727,...,355821595282,13513844769,51483262433593,4060603659405,8668278674,2022,2분기,005490,258000,1746110000741
81,20221114,POSCO홀딩스,00155319,-1.448500e+10,11014,87251112870,47402753865253,NaN,1607959638506,213864027182,...,356941976458,14401244987,50860490239671,3457736374418,9519018576,2022,3분기,005490,294500,1746110000741
82,20230309,POSCO홀딩스,00155319,-1.444721e+10,11011,44614715351,46840829094141,NaN,7649292423855,413709215105,...,359532593243,15901608719,51149119209287,4308290115146,10194840491,2022,4분기,005490,322500,1746110000741


# 2. PSR = 당기 공시기점의 종가 / (당기매출액 / 발행주식수) 

# api_result_csv 에 새로운 컬럼으로 붙일 것. 

# 당기 종가 * 발행주식수 / 매출총액

분기종가 * 발행주식수 / (매출총이익 + 매출원가)

In [3]:
api_final_result['PSR'] = None

for i in range(len(api_final_result)):
    mca = int(api_final_result.loc[i, '매출총이익'] + api_final_result.loc[i, '매출원가'])*1000
    api_final_result.loc[i, 'PSR'] = int(api_final_result.loc[i, '분기종가']) * int(api_final_result.loc[i, '발행주식수']) / mca
    
    
api_final_result

,기준일자,기업명,종목코드,유형자산의 처분,분기코드,이자의 수취,비유동자산,재고자산,매출총이익,비지배지분,...,무형자산,자산총계,유동자산,무형자산의 취득,사업년도,분기명,주가코드,분기종가,발행주식수,PSR
0,20180515,삼성전자,00126380,1.415690e+11,11013,94118000000,132836079000000,8.915860e+12,17410718000000,94546000000,...,2823356000000,205206164000000,72370085000000,218113000000,2018,1분기,005930,49200,1301110006246,1.502444
1,20180814,삼성전자,00126380,1.982810e+11,11012,222063000000,135899888000000,9.935909e+12,16708550000000,81090000000,...,2857695000000,203970112000000,68070224000000,429125000000,2018,2분기,005930,45150,1301110006246,1.421893
2,20181114,삼성전자,00126380,2.829450e+11,11014,360840000000,135895097000000,1.035734e+13,20067142000000,178867000000,...,2814997000000,217799444000000,81904347000000,604961000000,2018,3분기,005930,44100,1301110006246,1.204293
3,20190401,삼성전자,00126380,3.405580e+11,11011,459074000000,138981902000000,1.244095e+13,68715364000000,450142000000,...,2901476000000,219021357000000,80039455000000,880032000000,2018,4분기,005930,45050,1301110006246,0.344021
4,20190515,삼성전자,00126380,1.054600e+11,11013,98290000000,138335268000000,1.433641e+13,10586172000000,31372000000,...,2876176000000,220646957000000,82311689000000,250866000000,2019,1분기,005930,42550,1301110006246,1.494725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,20220516,POSCO홀딩스,00155319,-1.448500e+10,11013,38674284562,47458142280986,NaN,2928765425577,214947479400,...,13676722585,53333865909937,5875723628951,8668278674,2022,1분기,005490,281000,1746110000741,52.230744
80,20220816,POSCO홀딩스,00155319,-1.448500e+10,11012,62434274729,47422658774188,NaN,2733139872051,254454407727,...,13513844769,51483262433593,4060603659405,8668278674,2022,2분기,005490,258000,1746110000741,164.838879
81,20221114,POSCO홀딩스,00155319,-1.448500e+10,11014,87251112870,47402753865253,NaN,1607959638506,213864027182,...,14401244987,50860490239671,3457736374418,9519018576,2022,3분기,005490,294500,1746110000741,319.991546
82,20230309,POSCO홀딩스,00155319,-1.444721e+10,11011,44614715351,46840829094141,NaN,7649292423855,413709215105,...,15901608719,51149119209287,4308290115146,10194840491,2022,4분기,005490,322500,1746110000741,39.90214


In [4]:
# 망했다...gpa만 살리자

api_final_result['GP/A'] = None

for i in range(len(api_final_result)):
    gpa = int(api_final_result.loc[i, '매출총이익']) / int(api_final_result.loc[i, '자산총계'])
    api_final_result.loc[i, 'GP/A'] = gpa*100
    
    
api_final_result

,기준일자,기업명,종목코드,유형자산의 처분,분기코드,이자의 수취,비유동자산,재고자산,매출총이익,비지배지분,...,자산총계,유동자산,무형자산의 취득,사업년도,분기명,주가코드,분기종가,발행주식수,PSR,GP/A
0,20180515,삼성전자,00126380,1.415690e+11,11013,94118000000,132836079000000,8.915860e+12,17410718000000,94546000000,...,205206164000000,72370085000000,218113000000,2018,1분기,005930,49200,1301110006246,1.502444,8.4845
1,20180814,삼성전자,00126380,1.982810e+11,11012,222063000000,135899888000000,9.935909e+12,16708550000000,81090000000,...,203970112000000,68070224000000,429125000000,2018,2분기,005930,45150,1301110006246,1.421893,8.191666
2,20181114,삼성전자,00126380,2.829450e+11,11014,360840000000,135895097000000,1.035734e+13,20067142000000,178867000000,...,217799444000000,81904347000000,604961000000,2018,3분기,005930,44100,1301110006246,1.204293,9.213587
3,20190401,삼성전자,00126380,3.405580e+11,11011,459074000000,138981902000000,1.244095e+13,68715364000000,450142000000,...,219021357000000,80039455000000,880032000000,2018,4분기,005930,45050,1301110006246,0.344021,31.373819
4,20190515,삼성전자,00126380,1.054600e+11,11013,98290000000,138335268000000,1.433641e+13,10586172000000,31372000000,...,220646957000000,82311689000000,250866000000,2019,1분기,005930,42550,1301110006246,1.494725,4.797787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,20220516,POSCO홀딩스,00155319,-1.448500e+10,11013,38674284562,47458142280986,NaN,2928765425577,214947479400,...,53333865909937,5875723628951,8668278674,2022,1분기,005490,281000,1746110000741,52.230744,5.49138
80,20220816,POSCO홀딩스,00155319,-1.448500e+10,11012,62434274729,47422658774188,NaN,2733139872051,254454407727,...,51483262433593,4060603659405,8668278674,2022,2분기,005490,258000,1746110000741,164.838879,5.308793
81,20221114,POSCO홀딩스,00155319,-1.448500e+10,11014,87251112870,47402753865253,NaN,1607959638506,213864027182,...,50860490239671,3457736374418,9519018576,2022,3분기,005490,294500,1746110000741,319.991546,3.16151
82,20230309,POSCO홀딩스,00155319,-1.444721e+10,11011,44614715351,46840829094141,NaN,7649292423855,413709215105,...,51149119209287,4308290115146,10194840491,2022,4분기,005490,322500,1746110000741,39.90214,14.954886


In [5]:
api_final_result.to_csv('api_result_with_gpa.csv', index=False)